In [167]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import itertools

# from sklearn.preprocessing import StandardScaler    # 표준화.
# from sklearn.cluster import KMeans                  # 군집화

import warnings  
warnings.filterwarnings('ignore')

In [168]:
path = 'E:\기업프로젝트\profile_ymti_data_ver2'
path2 = 'E:\기업프로젝트\log_data_1027_2'
df = pd.read_csv(path+'\ymti_data.csv',engine='c',encoding='UTF-8')                                     # 원데이터
df_2030 = pd.read_csv(path + '\ymti_edit_data.csv',engine='c',encoding='UTF-8')                         # 20,30대 YMTI 데이터

want_df = pd.read_csv(path2 + '\want_log.csv',engine='c',encoding='UTF-8')                              # 만나고 싶어요 로그
concn_df = pd.read_csv(path2 + '\concn_log.csv',engine='c',encoding='UTF-8')                            # 관심있어요 로그
msg_df = pd.read_csv(path2 + '\msg_log.csv',engine='c',encoding='UTF-8')                                # 메세지 로그
pf_view_df = pd.read_csv(path2 + '\pf_view_log.csv',engine='c',encoding='UTF-8')                        # 프로필 열람 로그

married_df = pd.read_csv(path2 + '\married_success_sample.csv',engine='c',encoding='UTF-8')             # 결혼 성공 사례 로그

In [169]:
# df_2030 = df_2030[['mem_no','mem_id', 'mem_sex', 'mem_age', 'mem_loc', 'mem_birth_ddi',
#        'mate_blood', 'pf_ins_yn', 'mbti_code', 'food_cat']]
# df_2030

In [170]:
# a = df_2030['food_cat']
# print(a.shape, type(a))
# a

* 음식 카테고리별 추천
     - 컨텐츠 기반 필터링

In [171]:
# from ast import literal_eval

# df_2030['food_cat'] = df_2030['food_cat'].apply(literal_eval)

In [172]:
# df_2030['food_cat']

In [173]:
# df_2030['food_cat'] = df_2030['food_cat'].apply(lambda x : (' ').join(x) )
# df_2030

In [174]:
# type(df_2030['mem_no'][0])

In [175]:
# 피처 벡터 행렬 변환
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
food_mat = count_vect.fit_transform( df_2030['food_cat']  )
food_mat.shape

(9340, 30)

In [176]:
# 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity

food_sim = cosine_similarity(food_mat, food_mat)
print( food_mat.shape )
print( food_mat[:2] )

(9340, 30)
  (0, 17)	1
  (0, 18)	1
  (0, 22)	1
  (1, 18)	1


In [177]:
# 유사도가 높은 순으로 정리된 food_sim_sorted_ind 객체의 비교 행 위치 인덱스 값
food_sim_sorted_ind = food_sim.argsort()[:, ::-1]
food_sim_sorted_ind[:8]

array([[   0, 2170, 7791, ..., 3196, 8656, 8864],
       [4669, 5664, 2307, ..., 8869, 5319, 7543],
       [   0, 2170, 7791, ..., 3196, 8656, 8864],
       ...,
       [   0, 2170, 7791, ..., 3196, 8656, 8864],
       [9083, 6973, 8801, ..., 4140, 4136,    0],
       [9339, 2238, 2279, ..., 2666, 1664, 1041]], dtype=int64)

In [178]:
# 음식 카테고리에 따라 사람을 추천하는 함수를 생성    
def find_food_cat(df, sorted_ind, mem_id):
  title_mem = df_2030[ df_2030['mem_no'] == mem_id] 
  title_index = title_mem.index.values
  sim_indexs = sorted_ind[ title_index]

  sim_indexs = sim_indexs.reshape(-1)
  return df.iloc[sim_indexs]
      
def find_food_person(mem_id,top_n):
    a = df_2030[ df_2030['mem_no'] == mem_id]
    if a.mem_sex.values =="f":
        return find_food_cat(df_2030, food_sim_sorted_ind, mem_id )[find_food_cat(df_2030, food_sim_sorted_ind, mem_id )['mem_sex']=="m"][:top_n]
    elif a.mem_sex.values =="m":
        return find_food_cat(df_2030, food_sim_sorted_ind, mem_id )[find_food_cat(df_2030, food_sim_sorted_ind, mem_id )['mem_sex']=="f"][:top_n]

mem_id =1379275

find_food_person(mem_id,10)

,mem_no,mem_id,mem_sex,mem_age,mem_loc,mem_birth_ddi,mate_blood,pf_ins_yn,mbti_code,food_cat,concn
8509,1653250,koys,f,35,o,7,1,y,ESFP,korean chinese dessert western,0
6959,1605365,qZMT,f,25,f,5,2,y,ENFP,korean chinese dessert western,0
7889,1633197,q6aM,f,26,i,4,1,y,INTP,western korean chinese dessert,0
2077,1222404,YMZg,f,39,g,3,2,y,ESFJ,western korean chinese dessert,0
6121,1584336,h064,f,27,j,3,3,y,ISFJ,western korean chinese dessert,0
8010,1637287,n19e,f,24,h,6,1,y,ESFJ,western korean chinese dessert,0
7244,1613673,H2FC,f,35,i,7,3,y,ISFP,western korean chinese dessert,0
2018,1210143,J6mV,f,39,i,3,2,y,ISTP,western korean chinese dessert,0
4898,1555882,g4QF,f,37,b,5,2,y,ISFP,western korean chinese dessert,0
2358,1280225,CLEd,f,35,b,7,2,y,INFJ,western korean chinese dessert,0


* 로그 선호도별 사람 추천(INFP, ISFP)

In [179]:
df_2030

,mem_no,mem_id,mem_sex,mem_age,mem_loc,mem_birth_ddi,mate_blood,pf_ins_yn,mbti_code,food_cat,concn
0,6404,P80R,f,39,d,3,1,y,ENFP,korean japanese,0
1,8734,10Zs,m,33,g,9,1,y,INFJ,korean,0
2,28014,4sFm,m,38,k,4,1,y,ESFP,korean japanese,0
3,30302,BI03,m,39,b,3,1,y,INFP,western etc chinese korean,0
4,34189,9zZD,m,32,p,10,2,y,ISFP,western dessert japanese,0
...,...,...,...,...,...,...,...,...,...,...,...
9335,1687529,jUmc,m,36,p,6,1,y,ISTJ,western chinese dessert japanese,0
9336,1687672,8UKO,m,39,b,3,4,y,ISFP,western chinese korean japanese,0
9337,1687914,rvt5,m,33,c,9,2,y,INFJ,korean,0
9338,1687938,5kKC,m,32,b,10,1,n,INFP,western etc korean japanese,0


In [180]:
prefer = df_2030[(df_2030['mbti_code']=='INFP')| (df_2030['mbti_code']=='ISFP')]
prefer

,mem_no,mem_id,mem_sex,mem_age,mem_loc,mem_birth_ddi,mate_blood,pf_ins_yn,mbti_code,food_cat,concn
3,30302,BI03,m,39,b,3,1,y,INFP,western etc chinese korean,0
4,34189,9zZD,m,32,p,10,2,y,ISFP,western dessert japanese,0
6,43692,towo,m,39,b,3,2,y,INFP,western chinese,0
8,45401,TLXm,m,39,d,3,4,y,ISFP,dessert korean western chinese japanese,0
9,47565,SMUN,m,32,c,10,3,y,INFP,western korean japanese,0
...,...,...,...,...,...,...,...,...,...,...,...
9332,1687229,obtW,m,36,o,6,1,y,INFP,western japanese,0
9333,1687289,DSOe,m,36,f,6,1,y,ISFP,chinese dessert japanese,0
9336,1687672,8UKO,m,39,b,3,4,y,ISFP,western chinese korean japanese,0
9338,1687938,5kKC,m,32,b,10,1,n,INFP,western etc korean japanese,0


In [181]:
def find_log(mem_id,top_n):
    a = df_2030[df_2030['mem_no'] == mem_id]
    if a.mem_sex.values =="f":
        df_shuffled = prefer[prefer['mem_sex']=="m"].sample(frac=1)
        return(df_shuffled[:top_n])
        
    elif a.mem_sex.values =="m":
        df_shuffled = prefer[prefer['mem_sex']=="f"].sample(frac=1)
        return(df_shuffled[:top_n])

find_log(mem_id,10)


,mem_no,mem_id,mem_sex,mem_age,mem_loc,mem_birth_ddi,mate_blood,pf_ins_yn,mbti_code,food_cat,concn
4609,1542908,F3lq,f,33,l,9,1,y,INFP,western korean japanese,0
5271,1566491,Zxgb,f,34,j,8,2,y,INFP,japanese,0
8591,1656802,x7qY,f,29,b,1,3,y,ISFP,western korean dessert japanese,0
6705,1598569,dM2T,f,32,a,10,3,y,ISFP,etc korean japanese,0
3162,1409709,fBsZ,f,35,c,7,3,y,ISFP,western korean japanese,0
2710,1337634,FGXq,f,33,d,9,1,y,ISFP,chinese korean,0
4815,1552548,xD95,f,25,g,5,1,y,INFP,etc chinese korean japanese,0
2157,1241088,UDYF,f,34,i,8,1,y,ISFP,korean japanese,0
7157,1611188,ykQQ,f,28,o,2,4,y,INFP,western chinese korean japanese,0
4636,1545048,jfJL,f,30,i,0,3,y,INFP,western korean dessert japanese,0


* YMTI 상대추천(1:1 추천)
    1. 만나고싶어요
    2. 메세지
    3. 프로필
    4. 관심있어요

In [182]:
concn_df['concn']=int(1)
concn_df = concn_df[['mem_no','concn']]
concn_df

,mem_no,concn
0,1035596,1
1,1035596,1
2,1035596,1
3,1575626,1
4,1575626,1
...,...,...
14165,904319,1
14166,1638077,1
14167,909076,1
14168,1663346,1


In [183]:
df_2030['concn']=int(0)
df_2030 = df_2030.sort_values(by=['mem_no'],axis=0)
df_2030

,mem_no,mem_id,mem_sex,mem_age,mem_loc,mem_birth_ddi,mate_blood,pf_ins_yn,mbti_code,food_cat,concn
0,6404,P80R,f,39,d,3,1,y,ENFP,korean japanese,0
1,8734,10Zs,m,33,g,9,1,y,INFJ,korean,0
2,28014,4sFm,m,38,k,4,1,y,ESFP,korean japanese,0
3,30302,BI03,m,39,b,3,1,y,INFP,western etc chinese korean,0
4,34189,9zZD,m,32,p,10,2,y,ISFP,western dessert japanese,0
...,...,...,...,...,...,...,...,...,...,...,...
9335,1687529,jUmc,m,36,p,6,1,y,ISTJ,western chinese dessert japanese,0
9336,1687672,8UKO,m,39,b,3,4,y,ISFP,western chinese korean japanese,0
9337,1687914,rvt5,m,33,c,9,2,y,INFJ,korean,0
9338,1687938,5kKC,m,32,b,10,1,n,INFP,western etc korean japanese,0


In [184]:
df_inner = pd.merge(df_2030,concn_df,on='mem_no',suffixes=("_df2030", ""))
df_inner = df_inner.drop(['concn_df2030'],axis=1)

In [185]:
df_inner = df_inner.drop_duplicates()
df_inner

,mem_no,mem_id,mem_sex,mem_age,mem_loc,mem_birth_ddi,mate_blood,pf_ins_yn,mbti_code,food_cat,concn
0,145209,0lxc,f,31,b,11,1,y,ENTP,western dessert,1
12,152317,lg9H,m,34,b,8,1,y,ENFP,western korean,1
13,176254,WJW1,m,35,f,7,3,y,INFP,korean japanese,1
17,189181,MoVb,m,38,o,4,2,y,INFP,western etc chinese korean,1
18,219034,1wGa,m,38,c,4,4,y,INFP,western dessert japanese,1
...,...,...,...,...,...,...,...,...,...,...,...
3210,1685898,9SQi,m,35,b,7,3,y,ISFP,western chinese korean,1
3211,1686200,TZli,m,28,k,2,1,y,ENFJ,western,1
3212,1686346,ec7N,m,39,i,3,3,y,INFP,western chinese korean japanese,1
3213,1687047,Mcds,m,38,k,4,3,y,ESFJ,chinese dessert japanese,1


In [186]:
# df_2030['concn'] += df_inner['concn'] 

In [187]:
df_2030 = df_2030.fillna(int(0))
df_2030

,mem_no,mem_id,mem_sex,mem_age,mem_loc,mem_birth_ddi,mate_blood,pf_ins_yn,mbti_code,food_cat,concn
0,6404,P80R,f,39,d,3,1,y,ENFP,korean japanese,0
1,8734,10Zs,m,33,g,9,1,y,INFJ,korean,0
2,28014,4sFm,m,38,k,4,1,y,ESFP,korean japanese,0
3,30302,BI03,m,39,b,3,1,y,INFP,western etc chinese korean,0
4,34189,9zZD,m,32,p,10,2,y,ISFP,western dessert japanese,0
...,...,...,...,...,...,...,...,...,...,...,...
9335,1687529,jUmc,m,36,p,6,1,y,ISTJ,western chinese dessert japanese,0
9336,1687672,8UKO,m,39,b,3,4,y,ISFP,western chinese korean japanese,0
9337,1687914,rvt5,m,33,c,9,2,y,INFJ,korean,0
9338,1687938,5kKC,m,32,b,10,1,n,INFP,western etc korean japanese,0


In [188]:
new_data = df_2030.to_csv('E:\기업프로젝트\profile_ymti_data_ver2\ymti_edit_data.csv', index = None)

In [189]:
def find_ymti(mem_id,top_n):
    a = df_2030[df_2030['mem_no'] == mem_id]
    if a.mem_sex.values =="f":
        # 여자 일 때 - 남자 추천
        if a.mbti_code.values=="ISTJ":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="ESTP"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ISFJ":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="ESFP"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="INFJ":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="ENFP"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="INTJ":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="ENTP"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ISTP":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="ESTJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ISFP":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="ESFJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="INFP":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="ENFJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="INTP":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="ENTJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ESTP":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="ISTJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ESFP":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="ISFJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ENFP":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="INFJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ENTP":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="INTJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ESTJ":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="ISTP"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ESFJ":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="ISFP"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ENFJ":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="INFP"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ENTJ":
            b = df_2030[df_2030['mem_sex']=="m"]
            c = b[b['mbti_code']=="INTP"].sample(frac=1)
            return(c[:top_n])
           
    elif a.mem_sex.values =="m":
        # 남자 일 때 - 여자 추천
        if a.mbti_code.values=="ISTJ":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="ESTP"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ISFJ":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="ESFP"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="INFJ":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="ENFP"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="INTJ":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="ENTP"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ISTP":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="ESTJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ISFP":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="ESFJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="INFP":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="ENFJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="INTP":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="ENTJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ESTP":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="ISTJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ESFP":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="ISFJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ENFP":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="INFJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ENTP":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="INTJ"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ESTJ":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="ISTP"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ESFJ":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="ISFP"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ENFJ":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="INFP"].sample(frac=1)
            return(c[:top_n])
        elif a.mbti_code.values=="ENTJ":
            b = df_2030[df_2030['mem_sex']=="f"]
            c = b[b['mbti_code']=="INTP"].sample(frac=1)
            return(c[:top_n])

df_ymti = find_ymti(mem_id,30)
df_ymti = df_ymti.sort_values(by='concn', axis=0,ascending=False)
df_ymti

,mem_no,mem_id,mem_sex,mem_age,mem_loc,mem_birth_ddi,mate_blood,pf_ins_yn,mbti_code,food_cat,concn
3776,1478965,knUM,f,37,d,5,2,y,ENFJ,korean japanese,0
1065,982042,XUGS,f,39,z,3,2,y,ENFJ,western korean dessert,0
8632,1658354,qbq2,f,33,i,9,2,y,ENFJ,western etc korean japanese,0
8876,1666172,076Z,f,39,i,3,2,y,ENFJ,western etc korean japanese,0
1237,1023964,1ZGJ,f,39,n,3,3,y,ENFJ,western korean japanese,0
6665,1597503,RA9z,f,38,k,4,1,y,ENFJ,western chinese korean japanese,0
6725,1598998,iMFy,f,33,b,9,4,y,ENFJ,western dessert,0
3292,1423553,TsZp,f,29,e,1,2,y,ENFJ,western,0
5320,1567403,bfgB,f,28,d,2,1,y,ENFJ,chinese korean japanese,0
4605,1542660,uF9w,f,38,h,4,4,y,ENFJ,western korean,0


In [190]:
df_2030[df_2030['mem_no']==1379275]

,mem_no,mem_id,mem_sex,mem_age,mem_loc,mem_birth_ddi,mate_blood,pf_ins_yn,mbti_code,food_cat,concn
2991,1379275,Cx5c,m,35,i,7,4,y,INFP,korean chinese dessert western,0


* 추천 3명
    1. 음식
    2. 로그
    3. YMTI

In [191]:
df_2030

,mem_no,mem_id,mem_sex,mem_age,mem_loc,mem_birth_ddi,mate_blood,pf_ins_yn,mbti_code,food_cat,concn
0,6404,P80R,f,39,d,3,1,y,ENFP,korean japanese,0
1,8734,10Zs,m,33,g,9,1,y,INFJ,korean,0
2,28014,4sFm,m,38,k,4,1,y,ESFP,korean japanese,0
3,30302,BI03,m,39,b,3,1,y,INFP,western etc chinese korean,0
4,34189,9zZD,m,32,p,10,2,y,ISFP,western dessert japanese,0
...,...,...,...,...,...,...,...,...,...,...,...
9335,1687529,jUmc,m,36,p,6,1,y,ISTJ,western chinese dessert japanese,0
9336,1687672,8UKO,m,39,b,3,4,y,ISFP,western chinese korean japanese,0
9337,1687914,rvt5,m,33,c,9,2,y,INFJ,korean,0
9338,1687938,5kKC,m,32,b,10,1,n,INFP,western etc korean japanese,0


In [192]:
mem_id = 1687672

In [193]:
find_food_person(mem_id,1)

,mem_no,mem_id,mem_sex,mem_age,mem_loc,mem_birth_ddi,mate_blood,pf_ins_yn,mbti_code,food_cat,concn
7495,1620811,1Oxa,f,38,k,4,1,y,ENFP,western chinese korean japanese,0


In [194]:
find_log(mem_id,1)

,mem_no,mem_id,mem_sex,mem_age,mem_loc,mem_birth_ddi,mate_blood,pf_ins_yn,mbti_code,food_cat,concn
8352,1648372,LY2l,f,38,i,4,2,y,INFP,western,0


In [195]:
find_ymti(mem_id,1)

,mem_no,mem_id,mem_sex,mem_age,mem_loc,mem_birth_ddi,mate_blood,pf_ins_yn,mbti_code,food_cat,concn
2208,1250065,nlus,f,35,n,7,3,y,ESFJ,western chinese japanese,0
